# Scrape data from the Internet to create dataset

In [1]:
from bs4 import BeautifulSoup
import cloudscraper
import re
import timeit
import pandas as pd
from konoha import SentenceTokenizer

# a popular website to practice JLPT exams, contains many reading exercises and listening transcript
WEBSITE = "https://japanesetest4you.com/"

In [2]:
# to scrape website hosted by cloudflare
SCRAPER = cloudscraper.create_scraper()

def request_html(url) -> BeautifulSoup:
    response = SCRAPER.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

### Scrape reading exercise given url

In [3]:
def extract_reading_exercise(soup: BeautifulSoup):
    """
    Input:
    -   soup: from each reading exercise html

    Output:
    -   {
            'Reading Passage 1': [
                <p>学生: すみません。この本をかりたいです。<br/>としょかんの人: この学校の学生ですか。,
                <p>1. [_______] には何を入れますか。<br/><input name="quest1" type="radio" value="1"/>,
            ],
            'Reading Passage 2': [
                <p>リン「むらたさん、ちょっといいですか。」<br/>むらた「はい、なんですか。」,
                <p>5. ただしいものはどれですか。<br/><input name="quest5" type="radio" value="1"/>,
            ],
            'Answer Key:': [
                <p>Question 1: 1<br/>Question 2: 2<br/>Question 3: 4<br/>Question 4: 3<br/>Question 5: 4</p>,
            ],
            'JLPT N5 Kanji Lesson #3: 中 (chuu)': [
                <p><span>Download the video lesson and transcript</a></span></p>,
            ],
            'New words:': [
                <p>学生 (gakusei): student</p>,
                <p><span>Download the video lesson and transcript</span></p>,
            ],
        }
    """
    # get the body div
    passage_div = soup.find_all("div",class_= "entry clearfix")[0]
    content_nodes = [p for p in passage_div.children if p.name == 'p']

    # get the heading nodes (case-insensitive)
    passages_heading_nodes = [n for n in content_nodes if n.strong]

    # classify the texts according to heading
    sections = {}
    cur_heading = None
    for n in content_nodes:
        # make heading nodes key, and append subsequent contents in value
        if n in passages_heading_nodes:
            cur_heading = list(n.strings)[0].lower()
            sections[cur_heading] = []
        elif cur_heading is not None:
            sections[cur_heading].append(n)

    return sections

def sections_to_exercises(sections):
    """
    Input:
    -   sections: output of extract_reading_exercise()
    
    Output
    -   {
            'Reading Passage 1': {
                'passage': [
                    '学生: すみません。この本をかりたいです。',
                    'としょかんの人: この学校の学生ですか。',
                ],
                'questions': [
                    {
                        'question': '1. [_______] には何を入れますか。',
                        'options': [' 本は２９日、ざっしは２２日です。',' 本は２２日、ざっしは２９日です。',]
                    },
                ]
            },
        }
    """
    exercises = {}
    for section, contents in sections.items():
        # split reading passage into passage and questions
        if "reading passage" in section or "文章" in section:
            exercises[section] = {"passage": [], "questions": []}
            for content in contents:

                # check if content is problem or text
                problem_pattern = r"\d+\..*"
                if re.match(problem_pattern, list(content.strings)[0]):
                    strs = list(content.strings)
                    qna = {"question" : strs[0], "options": strs[1:]}
                    exercises[section]["questions"].append(qna)
                else:
                    exercises[section]["passage"] += list(content.strings)
        
        # TODO: add other section
    
    return exercises

**usage example**

In [4]:
soup = request_html("https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n5-reading-exercise-09/")

In [5]:
sections = extract_reading_exercise(soup)
exercises = sections_to_exercises(sections)
exercises

{'reading passage 1': {'passage': ['きのうは友だちのいえへ行って、友だちがつくったりょうりを食べました。すこしからかったです。でも、とてもおいしかったです。わたしも友だちもたくさん食べました。また食べたいです。'],
  'questions': [{'question': '1. ただしいものはどれですか。',
    'options': [' わたしはりょうりをつくりました。',
     ' 友だちはりょうりがじょうずです。',
     ' りょうりはおいしくありませんでした。',
     ' 友だちはあまり食べませんでした。']}]},
 'reading passage 2': {'passage': ['田中さんへ',
   'おとといはどうもありがとう。雨がたくさん降りましたから、田中さんに傘を借りてよかったです。借りた傘はドアのところに置きます。それから昨日作ったお菓子も置きます。どうぞ食べてください。',
   '９月１０日\u3000午後７時\u3000鈴木より'],
  'questions': [{'question': '2. 鈴木さんは９月１０日に何をしましたか。',
    'options': [' お菓子を買いました。', ' 傘を借りました。', ' 傘を返しました。', ' お菓子を作りました。']}]},
 'reading passage 3': {'passage': ['このたてものの中ではしずかにえを見てください。しゃしんをとらないでください。たばこもこまります。たばこは外ですってください。えを見ながら食べたり飲んだりしないでください。'],
  'questions': [{'question': '3. あなたはこのたてものの中で何をしますか。',
    'options': [' えを見ます。', ' たばこをすいます。', ' ごはんを食べます。', ' しゃしんをとります。']}]},
 'reading passage 4': {'passage': ['けさ、山川さんは７時におきました。あさごはんを食べる前にシャワーをあびました。あさごはんを食べながらテレビを見ました。あさごはんを食べたあとで、新聞を読みました。それから会社へ行きました。'],
  'quest

### Scrape links to reading exercise from menu

In [6]:
SCRAPER = cloudscraper.create_scraper()

def request_html(url) -> BeautifulSoup:
    response = SCRAPER.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

def extract_reading_exercise_urls(soup: BeautifulSoup):
    """
    Input:
    -   soup: from reading test menu in each jlpt level

    Output:
    -   [
            'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-1/#more-703',
            'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-2/#more-705',
        ]
    """
    links_leaf =soup.find_all("div", class_="readmore")
    links_to_exercise = [l.a.attrs["href"] for l in links_leaf]
    return links_to_exercise

**usage example**

In [7]:
soup = request_html(r"https://japanesetest4you.com/category/jlpt-n3/jlpt-n3-reading-test/")
links_to_exercise = extract_reading_exercise_urls(soup)
links_to_exercise

['https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-1/#more-703',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-2/#more-705',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-3/#more-708',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-4/#more-712',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-5/#more-715',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-6/#more-717',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-07/#more-2138',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-08/#more-2140',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-09/#more-2218',
 'https://japanesetest4you.com/japanese-language-

In [8]:
request_time = 0
extract_time = 0
to_exercise_time = 0

exercise_data = []
for link in links_to_exercise:
    start_time = timeit.default_timer()
    soup = request_html(link)

    ckpt1_time = timeit.default_timer()
    sections = extract_reading_exercise(soup)
    
    ckpt2_time = timeit.default_timer()
    exercises = sections_to_exercises(sections)

    end_time = timeit.default_timer()
    exercise_data.append(exercises)

    request_time += ckpt1_time - start_time
    extract_time += end_time - ckpt1_time
    to_exercise_time += end_time - ckpt2_time

print(f"Time for request: {request_time:.2f}s")
print(f"Time for extract: {extract_time:.2f}s")
print(f"Time for to_exercise: {to_exercise_time:.2f}s")

Time for request: 0.79s
Time for extract: 0.04s
Time for to_exercise: 0.00s


In [10]:
print(len(exercise_data))
exercise_data

16


[{'つぎの文章を読んで、質問に答えなさい。答えは、1・2・3・4から最もよいものを一つえらびなさい。': {'passage': [],
   'questions': []},
  'reading passage 1': {'passage': ['忘年会のご案内',
    '拝啓\u3000時下ますますご健勝のこととお喜び申し上げます。',
    'さて、この一年を振り返り、下記により忘年会を開催いたします。ご参加くださいますようお願いいたします。',
    '敬具',
    '記',
    '※参加を希望される方は12月20日までに弊社担当者にご連絡くださいますようお願いいたします。',
    '以上'],
   'questions': [{'question': '1.',
     'options': ['日時： 平成25年12月28日（土）\u3000午後15時～午後19時',
      '2. 場所： ミドリホテル',
      '東京都品川区広町1-2-3',
      '3.',
      '会費：\u3000お一人様\u30005000円',
      '4. 担当者： 山田太郎',
      '5. 連絡先： 0123-456-789']},
    {'question': '1. この案内書の内容と合っているものはどれか。',
     'options': [' 参加しない人が山田さんに連絡しなければならない。',
      ' 参加する人が山田さんに連絡しなければならない。',
      ' 参加しなくても会費を払わなければならない。',
      ' 参加する人が12月20日までに会費を払わなければならない。']}]},
  'reading passage 2': {'passage': ['好きな人と会話をするのなら、禁句（注1）にしたい言葉がある。それは「はい」と「いいえ」だ。例えば、「海外旅行に行ったことある？」という質問に「いいえ」で返してしまうと、終わりだ。別に相手の話に肯定も否定もするな、と言っているわけではない。「はい」と「いいえ」は使わず、別の表現に言い換えてみよう。「海外旅行に行ったことある？」という質問に「行ったことはないけど行ってみたい。一番行ってみたいのは中国。万里の長城

### Split Japanese text into sentences

In [12]:
sentence_tokenizer = SentenceTokenizer()
def split_to_sentences(text):
    sentences = sentence_tokenizer.tokenize(text)
    return sentences

In [13]:
text = "軍人役を任されてまず思ったことは、外見的に強く見せなければならないということでした。ユ・シジンは特殊戦司令部チーム長という役どころですから、まずは筋肉をつけるしかありません。また、強靭な印象を与えるために、ダイエットも並行していました。ロープ降下訓練、アクション訓練、射撃訓練、アルファチームとしての集団戦の訓練などさまざまな訓練を行いました。ナイフを使用して戦うため、剣術訓練を一番熱心にやりましたね。"
split_to_sentences(text)

['軍人役を任されてまず思ったことは、外見的に強く見せなければならないということでした。',
 'ユ・シジンは特殊戦司令部チーム長という役どころですから、まずは筋肉をつけるしかありません。',
 'また、強靭な印象を与えるために、ダイエットも並行していました。',
 'ロープ降下訓練、アクション訓練、射撃訓練、アルファチームとしての集団戦の訓練などさまざまな訓練を行いました。',
 'ナイフを使用して戦うため、剣術訓練を一番熱心にやりましたね。']

### Transform list of exercises to sentence dataset

In [18]:
"""
Input: 
- data: list of exercises
    [
        {
            'Reading Passage 1': {
                'passage': [
                    '学生: すみません。この本をかりたいです。',
                    'としょかんの人: この学校の学生ですか。',
                ],
                'questions': [
                    {
                        'question': '1. [_______] には何を入れますか。',
                        'options': [' 本は２９日、ざっしは２２日です。',' 本は２２日、ざっしは２９日です。',]
                    },
                ]
            },
        }
    ]
"""
sentences_dataset = {"N3": []}
texts = []
for exercise in exercise_data:
    for name, reading in exercise.items():
        # only passage and options are necessary
        texts += reading["passage"]
        for problem in reading["questions"]:
            texts += problem["options"]
            
# split all texts into sentences
for t in texts:
    sentences_dataset["N3"] += split_to_sentences(t)
df = pd.DataFrame(sentences_dataset)
df

,N3
0,忘年会のご案内
1,拝啓 時下ますますご健勝のこととお喜び申し上げます。
2,さて、この一年を振り返り、下記により忘年会を開催いたします。
3,ご参加くださいますようお願いいたします。
4,敬具
...,...
559,多くのドラマでは、台本が出てからその週に放送される２回分を撮影します。
560,撮影して３～４日後に放送する「___２___」こともあり、本当に撮影ができる人じゃないと間に...
561,台本をもらってそれを熟知する「___３___」大変なのに、それにプラスして感情など、すべての...
562,俳優やスタッフの徹夜作業も当たり前という環境でやってきましたが、今回は、明日、明後日に放送さ...


In [19]:
df.loc[2]["N3"]

'さて、この一年を振り返り、下記により忘年会を開催いたします。'